# 01 — Hedonic Price Index Prototype

Pull MLIT transaction data for Tokyo’s 23 wards and Sendai’s wards, clean and engineer features per the Step 1 plan, and extract a quarterly hedonic price index following Haque (2024).

## 1. Setup

In [1]:
from pathlib import Path
import requests
import pandas as pd
import numpy as np
import re

# Load MLIT API key from ../.env
MLIT_API_KEY = None
for line in Path('../.env').read_text().splitlines():
    if line.startswith('MLIT_API_KEY'):
        MLIT_API_KEY = line.split('=', 1)[1].strip().strip('"')
        break

assert MLIT_API_KEY, 'MLIT_API_KEY not found in ../.env'
HEADERS = {'Ocp-Apim-Subscription-Key': MLIT_API_KEY}

# Target time window (adjust as needed)
YEARS = list(range(2005, 2026))
QUARTERS = [1, 2, 3, 4]

## 2. Municipality Lists

In [ ]:
TOKYO_WARDS = [
    {'id': '13101', 'name': 'Chiyoda Ward'},
    {'id': '13102', 'name': 'Chuo Ward'},
    {'id': '13103', 'name': 'Minato Ward'},
    {'id': '13104', 'name': 'Shinjuku Ward'},
    {'id': '13105', 'name': 'Bunkyo Ward'},
    {'id': '13106', 'name': 'Taito Ward'},
    {'id': '13107', 'name': 'Sumida Ward'},
    {'id': '13108', 'name': 'Koto Ward'},
    {'id': '13109', 'name': 'Shinagawa Ward'},
    {'id': '13110', 'name': 'Meguro Ward'},
    {'id': '13111', 'name': 'Ota Ward'},
    {'id': '13112', 'name': 'Setagaya Ward'},
    {'id': '13113', 'name': 'Shibuya Ward'},
    {'id': '13114', 'name': 'Nakano Ward'},
    {'id': '13115', 'name': 'Suginami Ward'},
    {'id': '13116', 'name': 'Toshima Ward'},
    {'id': '13117', 'name': 'Kita Ward'},
    {'id': '13118', 'name': 'Arakawa Ward'},
    {'id': '13119', 'name': 'Itabashi Ward'},
    {'id': '13120', 'name': 'Nerima Ward'},
    {'id': '13121', 'name': 'Adachi Ward'},
    {'id': '13122', 'name': 'Katsushika Ward'},
    {'id': '13123', 'name': 'Edogawa Ward'},
]

SENDAI_WARDS = [
    {'id': '04101', 'name': 'Aoba Ward'},
    {'id': '04102', 'name': 'Miyagino Ward'},
    {'id': '04103', 'name': 'Wakabayashi Ward'},
    {'id': '04104', 'name': 'Taihaku Ward'},
    {'id': '04105', 'name': 'Izumi Ward'},
]

pd.DataFrame(TOKYO_WARDS + SENDAI_WARDS)


,id,name
0,13101,Chiyoda Ward
1,13102,Chuo Ward
2,13103,Minato Ward
3,13104,Shinjuku Ward
4,13105,Bunkyo Ward
5,13106,Taito Ward
6,13107,Sumida Ward
7,13108,Koto Ward
8,13109,Shinagawa Ward
9,13110,Meguro Ward


## 3. Pull Transactions (one request loop)

In [3]:
def fetch_transactions(city_code, year, quarter, language='en', retries=3):
    for attempt in range(retries):
        try:
            response = requests.get(
                'https://www.reinfolib.mlit.go.jp/ex-api/external/XIT001',
                headers=HEADERS,
                params={
                    'priceClassification': '01',
                    'year': year,
                    'quarter': quarter,
                    'city': city_code,
                    'language': language,
                },
                timeout=60
            )
            response.raise_for_status()
            return response.json().get('data', [])
        except requests.HTTPError as exc:
            if attempt == retries - 1:
                print(f"{city_code} {year}Q{quarter} failed: {exc}")
                return []
        except requests.RequestException as exc:
            if attempt == retries - 1:
                print(f"{city_code} {year}Q{quarter} error: {exc}")
                return []

batch_records = []
municipalities = TOKYO_WARDS + SENDAI_WARDS
year_batches = [list(range(start, min(start + 5, YEARS[-1] + 1))) for start in range(YEARS[0], YEARS[-1] + 1, 5)]

for batch_years in year_batches:
    print(f"Pulling years {batch_years[0]}-{batch_years[-1]}...")
    part_records = []
    for muni in municipalities:
        for year in batch_years:
            for quarter in QUARTERS:
                rows = fetch_transactions(muni['id'], year, quarter)
                for row in rows:
                    row['CityCode'] = muni['id']
                    row['AreaLabel'] = muni['name']
                    row['YearParam'] = year
                    row['QuarterParam'] = quarter
                    part_records.append(row)
    if part_records:
        batch_df = pd.DataFrame(part_records)
        print(f"  • {len(batch_df):,} rows in this batch")
        batch_records.append(batch_df)

transactions_df = pd.concat(batch_records, ignore_index=True)
print(f"Transactions pulled (raw): {len(transactions_df):,}")
print(f"Columns returned: {sorted(transactions_df.columns.tolist())}")
transactions_df.head()


Pulling years 2005-2009...
13101 2005Q1 failed: 400 Client Error: Bad Request for url: https://www.reinfolib.mlit.go.jp/ex-api/external/XIT001?priceClassification=01&year=2005&quarter=1&city=13101&language=en
13101 2005Q2 failed: 400 Client Error: Bad Request for url: https://www.reinfolib.mlit.go.jp/ex-api/external/XIT001?priceClassification=01&year=2005&quarter=2&city=13101&language=en
13102 2005Q1 failed: 400 Client Error: Bad Request for url: https://www.reinfolib.mlit.go.jp/ex-api/external/XIT001?priceClassification=01&year=2005&quarter=1&city=13102&language=en
13102 2005Q2 failed: 400 Client Error: Bad Request for url: https://www.reinfolib.mlit.go.jp/ex-api/external/XIT001?priceClassification=01&year=2005&quarter=2&city=13102&language=en
13103 2005Q1 failed: 400 Client Error: Bad Request for url: https://www.reinfolib.mlit.go.jp/ex-api/external/XIT001?priceClassification=01&year=2005&quarter=1&city=13103&language=en
13103 2005Q2 failed: 400 Client Error: Bad Request for url: htt

,PriceCategory,Type,Region,MunicipalityCode,Prefecture,Municipality,DistrictName,TradePrice,PricePerUnit,FloorPlan,...,CityPlanning,CoverageRatio,FloorAreaRatio,Period,Renovation,Remarks,CityCode,AreaLabel,YearParam,QuarterParam
0,Real Estate Transaction Price Information,Residential Land(Land and Building),Commercial Area,13101,Tokyo,Chiyoda Ward,Sotokanda,230000000,,,...,Commercial Zone,80,600,3rd quarter 2005,,,13101,Chiyoda Ward,2005,3
1,Real Estate Transaction Price Information,Residential Land(Land and Building),Commercial Area,13101,Tokyo,Chiyoda Ward,Kandaizumicho,120000000,,,...,Commercial Zone,80,500,3rd quarter 2005,,,13101,Chiyoda Ward,2005,3
2,Real Estate Transaction Price Information,Residential Land(Land Only),Commercial Area,13101,Tokyo,Chiyoda Ward,Kandasakumacho,6800000000,14000000,,...,Commercial Zone,80,800,3rd quarter 2005,,,13101,Chiyoda Ward,2005,3
3,Real Estate Transaction Price Information,Residential Land(Land and Building),Commercial Area,13101,Tokyo,Chiyoda Ward,Kandasakumacho,150000000,,,...,Commercial Zone,80,500,3rd quarter 2005,,,13101,Chiyoda Ward,2005,3
4,Real Estate Transaction Price Information,"Pre-owned Condominiums, etc.",,13101,Tokyo,Chiyoda Ward,Kudamminami,20000000,,1LDK,...,Commercial Zone,80,500,3rd quarter 2005,Done,,13101,Chiyoda Ward,2005,3


## 4. Clean Numeric Fields

In [4]:
def parse_price(value):
    if pd.isna(value) or value == '':
        return np.nan
    value = str(value).replace(',', '').strip()
    if value.isdigit():
        return float(value)
    if '-' in value:
        parts = [p for p in value.split('-') if p.strip().isdigit()]
        if parts:
            return sum(float(p) for p in parts) / len(parts)
    return np.nan


def parse_area(value):
    if pd.isna(value) or value == '':
        return np.nan
    value = str(value).replace(',', '').strip()
    try:
        return float(value)
    except ValueError:
        return np.nan

QUARTER_PATTERN = re.compile(r'(\d)(?:st|nd|rd|th) quarter (\d{4})')

def parse_period(period_text, fallback_year, fallback_quarter):
    if pd.isna(period_text):
        return f"{fallback_year}-Q{fallback_quarter}"
    match = QUARTER_PATTERN.search(str(period_text))
    if match:
        return f"{match.group(2)}-Q{match.group(1)}"
    return f"{fallback_year}-Q{fallback_quarter}"

transactions_df['TradePriceValue'] = transactions_df['TradePrice'].apply(parse_price)
transactions_df['AreaSqM'] = transactions_df['Area'].apply(parse_area)
transactions_df['PeriodKey'] = transactions_df.apply(
    lambda row: parse_period(row.get('Period'), row['YearParam'], row['QuarterParam']), axis=1
)
transactions_df['PricePerSqM'] = transactions_df['TradePriceValue'] / transactions_df['AreaSqM']

clean_df = transactions_df.dropna(subset=['TradePriceValue', 'AreaSqM', 'PeriodKey']).copy()
clean_df.head()


,PriceCategory,Type,Region,MunicipalityCode,Prefecture,Municipality,DistrictName,TradePrice,PricePerUnit,FloorPlan,...,Renovation,Remarks,CityCode,AreaLabel,YearParam,QuarterParam,TradePriceValue,AreaSqM,PeriodKey,PricePerSqM
0,Real Estate Transaction Price Information,Residential Land(Land and Building),Commercial Area,13101,Tokyo,Chiyoda Ward,Sotokanda,230000000,,,...,,,13101,Chiyoda Ward,2005,3,2.300000e+08,90.0,2005-Q3,2.555556e+06
1,Real Estate Transaction Price Information,Residential Land(Land and Building),Commercial Area,13101,Tokyo,Chiyoda Ward,Kandaizumicho,120000000,,,...,,,13101,Chiyoda Ward,2005,3,1.200000e+08,95.0,2005-Q3,1.263158e+06
2,Real Estate Transaction Price Information,Residential Land(Land Only),Commercial Area,13101,Tokyo,Chiyoda Ward,Kandasakumacho,6800000000,14000000,,...,,,13101,Chiyoda Ward,2005,3,6.800000e+09,1600.0,2005-Q3,4.250000e+06
3,Real Estate Transaction Price Information,Residential Land(Land and Building),Commercial Area,13101,Tokyo,Chiyoda Ward,Kandasakumacho,150000000,,,...,,,13101,Chiyoda Ward,2005,3,1.500000e+08,120.0,2005-Q3,1.250000e+06
4,Real Estate Transaction Price Information,"Pre-owned Condominiums, etc.",,13101,Tokyo,Chiyoda Ward,Kudamminami,20000000,,1LDK,...,Done,,13101,Chiyoda Ward,2005,3,2.000000e+07,20.0,2005-Q3,1.000000e+06


In [5]:
transactions_df.to_csv("transactions.csv", index=False)

In [6]:
# if using csv directly
transactions_df = pd.read_csv("transactions.csv")
transactions_df.head()

C:\Users\ignit\AppData\Local\Temp\ipykernel_25968\3180769579.py:2: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  transactions_df = pd.read_csv("transactions.csv")


,PriceCategory,Type,Region,MunicipalityCode,Prefecture,Municipality,DistrictName,TradePrice,PricePerUnit,FloorPlan,...,Renovation,Remarks,CityCode,AreaLabel,YearParam,QuarterParam,TradePriceValue,AreaSqM,PeriodKey,PricePerSqM
0,Real Estate Transaction Price Information,Residential Land(Land and Building),Commercial Area,13101,Tokyo,Chiyoda Ward,Sotokanda,230000000,NaN,NaN,...,NaN,NaN,13101,Chiyoda Ward,2005,3,2.300000e+08,90.0,2005-Q3,2.555556e+06
1,Real Estate Transaction Price Information,Residential Land(Land and Building),Commercial Area,13101,Tokyo,Chiyoda Ward,Kandaizumicho,120000000,NaN,NaN,...,NaN,NaN,13101,Chiyoda Ward,2005,3,1.200000e+08,95.0,2005-Q3,1.263158e+06
2,Real Estate Transaction Price Information,Residential Land(Land Only),Commercial Area,13101,Tokyo,Chiyoda Ward,Kandasakumacho,6800000000,14000000.0,NaN,...,NaN,NaN,13101,Chiyoda Ward,2005,3,6.800000e+09,1600.0,2005-Q3,4.250000e+06
3,Real Estate Transaction Price Information,Residential Land(Land and Building),Commercial Area,13101,Tokyo,Chiyoda Ward,Kandasakumacho,150000000,NaN,NaN,...,NaN,NaN,13101,Chiyoda Ward,2005,3,1.500000e+08,120.0,2005-Q3,1.250000e+06
4,Real Estate Transaction Price Information,"Pre-owned Condominiums, etc.",NaN,13101,Tokyo,Chiyoda Ward,Kudamminami,20000000,NaN,1LDK,...,Done,NaN,13101,Chiyoda Ward,2005,3,2.000000e+07,20.0,2005-Q3,1.000000e+06
